##  Topic Modelling 
As the name suggests, it is a process to automatically identify topics present in a text object and to derive hidden patterns exhibited by a text corpus.

Topic Modelling is different from rule-based text mining (regex) . It is an unsupervised approach used for finding and observing the bunch of words (called “topics”) in large clusters of texts.

Topics can be defined as “a repeating pattern of co-occurring terms in a corpus”. A good topic model should result in – “health”, “doctor”, “patient”, “hospital” for a topic – Healthcare, and “farm”, “crops”, “wheat” for a topic – “Farming”.

More formally, we define a **topic** to be a distribution over a fixed vocabulary.



## Use Cases

New York Times are using topic models to boost their user – article recommendation engines. Various corporations are using topic models for recruitment industries where they aim to extract latent features of job descriptions and map them to right candidates. 

## Latent Dirichlet Allocation (LDA)

A popular topic modeling technique, LDA assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. Given a dataset of documents, LDA backtracks and tries to figure out what topics would create those documents in the first place --> THIS OUR LIKELIHOOD

In [81]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

## Sklearn Implementation

In [82]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model


tf_vectorizer = CountVectorizer(max_features=10, stop_words='english')
tf = tf_vectorizer.fit_transform(doc_complete)
tf_feature_names = tf_vectorizer.get_feature_names()

In [83]:
tf_feature_names

['bad',
 'driving',
 'father',
 'pressure',
 'say',
 'school',
 'sister',
 'spends',
 'stress',
 'sugar']

### Discussion

Min_df: ignore terms that have a document frequency strictly lower than the given threshold.
Max_df: ignore terms that have a document frequency strictly higher than the given threshold.

If integer is passed occurences are considered, if float b/w 0.0 and 1.0 then frequency. 

In [84]:
tf_feature_names

['bad',
 'driving',
 'father',
 'pressure',
 'say',
 'school',
 'sister',
 'spends',
 'stress',
 'sugar']

In [97]:
from sklearn.decomposition import LatentDirichletAllocation
# Run LDA
lda = LatentDirichletAllocation(n_topics=2, max_iter=5).fit(tf)


/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [99]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, 3)

Topic 0:
pressure say stress
Topic 1:
sister father sugar


## Gensim and By hand cleaning Implementation

In [93]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #normalized = set(normalized)
    print(normalized)
    print("********")
    print(doc)
    print("\n")
    
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]

sugar bad consume sister like sugar father
********
Sugar is bad to consume. My sister likes to have sugar, but not my father.


father spends lot time driving sister around dance practice
********
My father spends a lot of time driving my sister around to dance practice.


doctor suggest driving may cause increased stress blood pressure
********
Doctors suggest that driving may cause increased stress and blood pressure.


sometimes feel pressure perform well school father never seems drive sister better
********
Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better.


health expert say sugar good lifestyle
********
Health experts say that sugar is not good for your lifestyle.




In [88]:
doc_clean = [set (i) for i in doc_clean]

In [57]:
doc_clean

[{'bad', 'consume', 'father', 'like', 'sister', 'sugar'},
 {'around',
  'dance',
  'driving',
  'father',
  'lot',
  'practice',
  'sister',
  'spends',
  'time'},
 {'blood',
  'cause',
  'doctor',
  'driving',
  'increased',
  'may',
  'pressure',
  'stress',
  'suggest'},
 {'better',
  'drive',
  'father',
  'feel',
  'never',
  'perform',
  'pressure',
  'school',
  'seems',
  'sister',
  'sometimes',
  'well'},
 {'expert', 'good', 'health', 'lifestyle', 'say', 'sugar'}]

In [58]:
import sys
!conda install --yes --prefix {sys.prefix} gensim

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.4

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [59]:
# Importing Gensim. A module specifically meant for Topic Modelling 

import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)


# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [60]:
print(dictionary)

Dictionary(35 unique tokens: ['bad', 'consume', 'father', 'like', 'sister']...)


In [61]:
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(2, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(8, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1)],
 [(2, 1),
  (4, 1),
  (18, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)],
 [(5, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]]

In [94]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word = dictionary, passes=50)

In [96]:
print(ldamodel.print_topics(num_topics=2, num_words=3))

[(0, '0.043*"sugar" + 0.042*"pressure" + 0.042*"father"'), (1, '0.060*"driving" + 0.060*"sister" + 0.060*"father"')]


In [ ]:
Topic 0:
pressure say stress
Topic 1:
sister father sugar